In [1]:
import pandas as pd
import statsmodels.api as sm


def forward_regression(X, y,
                       threshold_in,
                       verbose=False):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        if not changed:
            break

    return included

def backward_regression(X, y,
                           threshold_out,
                           verbose=False):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
df = pd.read_csv('./data/TP_eng.csv')

In [30]:
X = df.drop(columns=['point','date','dx','dy'])
delta_x = df['dx']
delta_y = df['dy']

In [31]:
threshold_in = 0.05
included = backward_regression(X, delta_x,
                       threshold_in,
                       verbose=True)
print(included)

Drop h2                             with p-value 0.93163
Drop h3                             with p-value 0.895027
Drop h4                             with p-value 0.360635
Drop T5                             with p-value 0.0909548
['h', 'T', 'T20', 'T15', 'T10', 'day', 'day1', 'day2']


In [32]:
X_update = X[included]

In [37]:
included2 = forward_regression(X_update, delta_x,
                       threshold_in,
                       verbose=True)
print(included2)

Add  T                              with p-value 0.0
Add  day                            with p-value 0.0
Add  h                              with p-value 8.53398e-204
Add  day1                           with p-value 6.32567e-42
Add  day2                           with p-value 2.97007e-06
Add  T10                            with p-value 0.0270008
Add  T15                            with p-value 0.0055513
Add  T20                            with p-value 0.00916981
['T', 'day', 'h', 'day1', 'day2', 'T10', 'T15', 'T20']


In [40]:
if included.sort() == included2.sort():
    print('algoithm ended with following selected features:\n',included)

algoithm ended with following selected features:
 ['T', 'T10', 'T15', 'T20', 'day', 'day1', 'day2', 'h']


In [36]:
included

['T', 'day', 'h', 'day1', 'day2', 'T10', 'T15', 'T20']